This vignette is part of the workshop: [Hands-on Tour of the Visium Spatial Gene Expression Analysis Journey](https://www.10xgenomics.com/analysis-guides/workshop-visium-hd-analysis).

We will begin by installing the necessary packages.

**Option #1 (may take over 20 minutes):** Install packages directly from R repositories.

In [1]:
install.packages("remotes")
install.packages("devtools")
system("apt install libgsl-dev", intern=TRUE)
devtools::install_github("paulponcet/lplyr")

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = '3.21',ask = FALSE)

BiocManager::install("clusterProfiler")
BiocManager::install("enrichplot")
BiocManager::install("ggplot2")
BiocManager::install("msigdbr")
BiocManager::install("dplyr")
BiocManager::install("DOSE")
BiocManager::install("forcats")
BiocManager::install("AnnotationDbi")
BiocManager::install("org.Hs.eg.db")
install.packages("msigdbdf", repos = "https://igordot.r-universe.dev")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



[1] "Reading package lists..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [2] "Building dependency tree..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 [3] "Reading state information..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [4] "The following additional packages will be installed:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [5] "  libgsl27 libgslcblas0"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 [6] "Suggested packages:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
 [7] "  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html"                                                           

lobstr   (NA -> 1.1.2 ) [CRAN]
pryr     (NA -> 0.1.6 ) [CRAN]
plyr     (NA -> 1.8.9 ) [CRAN]
kimisc   (NA -> 1.0.0 ) [CRAN]
lazyeval (NA -> 0.2.2 ) [CRAN]
bazar    (NA -> 1.0.11) [CRAN]


Installing 6 packages: lobstr, pryr, plyr, kimisc, lazyeval, bazar

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘pryr’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘kimisc’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘bazar’ had non-zero exit status”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp3UQM4I/remotesbb150fccf6/paulponcet-lplyr-51a576b/DESCRIPTION’ ... OK
* preparing ‘lplyr’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘lplyr_0.1.12.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘/tmp/Rtmp3UQM4I/filebb3f24f6d2/lplyr_0.1.12.tar.gz’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-11)

Installing package(s) 'BiocVersion'

Old packages: 'data.table', 'zip'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-11)

Installing package(s) 'clusterProfiler'

also installing the dependencies ‘UCSC.utils’, ‘GenomeInfoDb

**Option #2 (recommended for in-person workshop):** Install packages by directly downloading pre-installed packages from AWS s3 bucket

In [ ]:
# Download the pre-installed R packages:
download.file("https://cf.10xgenomics.com/supp/spatial-exp/analysis-workshop/R_packages.tar.gz", "R_packages.tar.gz")


# move all pre-installed R packages to default R folder path
system("tar -xzvf /content/R_packages.tar.gz")
system(paste0("mv /content/R_packages/* ", .libPaths()[1]))

Next, load these required packages into the environment.

In [ ]:
library(clusterProfiler)
library(enrichplot)
library(ggplot2)
library(msigdbr)
library(dplyr)
library(lplyr)
library(DOSE)
library(forcats)
library(AnnotationDbi)
library(org.Hs.eg.db)
library(msigdbdf)

Read the in CSV file containing gene expression information and take a look at the dataframe

In [ ]:
download.file("https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/Visium_HD_GSEA/ROI_Features.csv", "ROI_Features.csv")
df <- read.csv("ROI_Features.csv", header = TRUE)
head(df)

How many genes are in our list?

In [ ]:
nrow(df)

GSEA requires Entrez IDs as input. These are unique integer identifiers for genes from NCBI. We will use AnnotationDbi to find the IDs for each gene symbol in our list and take a look at the top of the data frame to see what it looks like.

In [ ]:
entrez_data <- AnnotationDbi::select(org.Hs.eg.db, keys = df$SYMBOL,columns = c("SYMBOL", "ENTREZID"),keytype = "SYMBOL")
head(entrez_data)

Not every gene symbol has an associated Entrez ID. We will need to remove genes that do not have an Entrez ID ("NA") from our input list.

In [ ]:
anno_result <- entrez_data %>%
  filter(!is.na(ENTREZID)) %>%
  inner_join(df, by = "SYMBOL",relationship = "many-to-many")

anno_result_unique <- anno_result %>%
  group_by(ENTREZID) %>%
  summarize(Log2FC = mean(Log2FC))

head(anno_result_unique)

How many genes remain after filtering NAs?

In [ ]:
nrow(anno_result_unique)

The starting gene number was 18,072. So, we removed 437 or ~2% of our gene list due to NAs. This is not a large proportion of our gene list and so we will proceed assuming the list is still representative of our sample.

Next, we will turn this data frame into a named vector that has the Entrez ID and associated Log2FC information and sort it largest Log2FC to smallest.

In [ ]:
geneList <- with(anno_result_unique, setNames(Log2FC, ENTREZID))
geneList <- sort(geneList, decreasing = TRUE)

Next, we need to prepare our reference gene sets that we will use for analysis. There are a number of databases available to use. We chose to use Cancer Gene Networks from the Computational Gene Sets. We can access the gene sets in this database with the msigdbr package from MSigDB. To learn more about the gene sets available through this resource, go to this web page: https://www.gsea-msigdb.org/gsea/msigdb/human/collections.jsp

We find there is a database of computational gene sets defined by expression neighborhoods centered around cancer-associated genes. This database is relevant to our data, so we will use this one.

In [ ]:
c4_t2g <- msigdbr(species = "Homo sapiens", category = "C4", subcollection = "CGN") %>%
  dplyr::select(gs_name, entrez_gene)
head(c4_t2g)

Now that we have our ranked gene list and gene set reference inputs, we can run GSEA.

In [ ]:
c4 <- GSEA(geneList, TERM2GENE = c4_t2g)
c4

The gene information is stored as Entrez ID, so we need to add the gene symbols into the c4 object to be able to interpret the results better. We will also put our results into a dataframe so they are easier to navigate.

In [ ]:
c4 <- setReadable(c4, 'org.Hs.eg.db', 'ENTREZID')
c4_df <- c4@result

Our analysis is complete and stored in an object we called “c4”. There were ~170-180 enriched terms found in our data. Results may vary slightly due to random seeding.

We can use the object storing the results to generate some plots of our results. First, we will take a look at the overall summary results. We can visualize the top 10 gene sets that were up-regulated (activated in microenvironment), meaning these gene sets were enriched at the top of our list, where up-regulated genes were located. And we will also visualize the top 10 gene sets that were down-regulated (suppressed in microenvironment), meaning these gene sets were enriched at the bottom of our list, where down-regulated genes were located.

In [ ]:
#set plot size
options(repr.plot.width=10, repr.plot.height=6)

#sort data and assign to "activated" or "suppressed" groups based on NES
sorted_c4<- c4@result[order(c4@result$NES, decreasing = F),]
sorted_c4$color<-ifelse(sorted_c4$NES<0, "Enriched in tumor", "Enriched in TME")

#plot results
sorted_c4 %>%
  dplyr::group_by(color) %>%
  dplyr::arrange(desc(abs(NES))) %>%
  slice_head(n = 10) %>%
  ggplot(aes(x = NES, y = reorder(Description, NES), fill = color)) +
  geom_bar(stat = "identity") +
  geom_vline(xintercept = 0) +
  labs(y = "Description") +
  theme_classic() +
  scale_fill_manual(values=c("#ffa557", "#4296f5")) +
  theme(legend.position = "right")

#save a publicaiton-quality version of the plot
ggsave("go_barplot.pdf",
        dpi = 600,
        width = 30, height = 15, unit = "cm")

Next, we can select one of these gene sets to explore more in-depth.


The database we are using organizes the gene pathways into sets of genes centered around known cancer-related genes. We will need to make use of the GSEA website to understand what each gene set is.

For example, looking into one of the modules enriched in the microenvironment region, [GNF2_PECAM1](https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/GNF2_PECAM1.html), we can learn more about it. First, we can take a look at the full list of genes in this gene set.

Based on the literature, we may find promising cancer treatment targets by looking at the genes in this gene set. We can generate a GSEA plot and a cnet plot for this module to evaluate the full set of genes in our data:

In [ ]:
#re-set plot size
options(repr.plot.width=8, repr.plot.height=6)

#gsea plot
gseaplot(c4, by = "all", title = "PECAM1 Network", geneSetID = "GNF2_PECAM1")

And pull out the NES and adjusted p-value associated with this enriched set.

In [ ]:
c4_df["GNF2_PECAM1",]

Of the 56 genes in this PECAM1 Gene Set, 33 were detected in our experiment. We can visualize these genes in the cnet plot below:


In [ ]:
cnetplot(c4, categorySize="pvalue", showCategory = c("GNF2_PECAM1"))

Platelet endothelial cell adhesion molecule (PECAM-1) is a cell-cell adhesion protein found on endothelial cells, platelets, macrophages and Kupffer cells, granulocytes, lymphocytes (T cells, B cells, and NK cells), megakaryocytes, and osteoclasts.Going to the literature, this gene has been studied in the cancer field. Specifically, [one study](https://doi.org/10.1073/pnas.1004654107) found that PECAM-1 in the tumor microenvironment drives advanced metastatic progression of tumor cells. [Another](https://doi.org/10.1023/A:1009092107382) found that an antibody against murine PECAM-1 inhibits tumor angiogenesis in mice.

We may find promising cancer treatment targets by looking at the genes in this gene set. The PECAM1 neighborhood identified with GSEA may represent a network of genes that are involved in regulation of the the tumor microenvironment to make the area more susceptible to tumor invasion. These genes may be worth investigating in future studies and specifically targeting in future cancer treatment development. The last thing we will do is go back to Loupe Browser to visualize the expression of these genes in our tissue.

In [ ]:
sessionInfo()